In [1]:
import sys
import subprocess

print("🧹 Cleaning up conflicting packages and fixing environment...")

python_exe = sys.executable

# 1. UNINSTALL conflicts first
# We remove 'earsegmentationai' and 'sam-2' because they enforce broken dependency rules.
# We also remove numpy/torch to ensure a clean re-link.
uninstall_cmd = [
    python_exe, "-m", "pip", "uninstall", "-y",
    "earsegmentationai",
    "sam-2",
    "torch",
    "torchvision",
    "torchaudio",
    "numpy" 
]

print(f"   EXEC: {' '.join(uninstall_cmd)}")
try:
    subprocess.check_call(uninstall_cmd)
except subprocess.CalledProcessError:
    print("   ⚠️ Warning: Some packages were not found (this is fine).")

# 2. INSTALL NUMPY (CRITICAL FIX)
# We strictly pin numpy < 2.0.0 because PyTorch 2.0.1 crashes with NumPy 2.x
# We install this FIRST so Torch links against it correctly.
install_numpy_cmd = [
    python_exe, "-m", "pip", "install",
    "numpy<2.0.0",
    "--no-cache-dir"
]

print(f"\n⬇️ INSTALLING NUMPY: {' '.join(install_numpy_cmd)}")
subprocess.check_call(install_numpy_cmd)

# 3. INSTALL the correct Torch stack (Torch 2.0.1 + CUDA 11.7)
install_torch_cmd = [
    python_exe, "-m", "pip", "install",
    "torch==2.0.1",
    "torchvision==0.15.2",
    "torchaudio==2.0.2",
    "--index-url", "https://download.pytorch.org/whl/cu117",
    "--no-cache-dir"
]

print(f"\n⬇️ INSTALLING TORCH: {' '.join(install_torch_cmd)}")
subprocess.check_call(install_torch_cmd)

# 4. INSTALL required libraries
# CRITICAL: We add "numpy<2.0.0" here again to prevent ultralytics/scipy from upgrading it
install_deps_cmd = [
    python_exe, "-m", "pip", "install",
    "numpy<2.0.0",
    "ultralytics",
    "opencv-python",
    "matplotlib",
    "scipy",
    "numpy-stl",
    "segment-anything"
]

print(f"\n⬇️ INSTALLING DEPS: {' '.join(install_deps_cmd)}")
subprocess.check_call(install_deps_cmd)

print("\n✅ Environment Repair Complete!")
print("📢 PLEASE RESTART THE KERNEL NOW (Kernel > Restart) before running your code.")

🧹 Cleaning up conflicting packages and fixing environment...
   EXEC: a:\PROJECT\AUTO MICROTIA\Ear-segmentation-ai\ear_seg_env.venv\Scripts\python.exe -m pip uninstall -y earsegmentationai sam-2 torch torchvision torchaudio numpy

⬇️ INSTALLING NUMPY: a:\PROJECT\AUTO MICROTIA\Ear-segmentation-ai\ear_seg_env.venv\Scripts\python.exe -m pip install numpy<2.0.0 --no-cache-dir

⬇️ INSTALLING TORCH: a:\PROJECT\AUTO MICROTIA\Ear-segmentation-ai\ear_seg_env.venv\Scripts\python.exe -m pip install torch==2.0.1 torchvision==0.15.2 torchaudio==2.0.2 --index-url https://download.pytorch.org/whl/cu117 --no-cache-dir

⬇️ INSTALLING DEPS: a:\PROJECT\AUTO MICROTIA\Ear-segmentation-ai\ear_seg_env.venv\Scripts\python.exe -m pip install numpy<2.0.0 ultralytics opencv-python matplotlib scipy numpy-stl segment-anything

✅ Environment Repair Complete!
📢 PLEASE RESTART THE KERNEL NOW (Kernel > Restart) before running your code.


In [37]:
#pip install samgeo
#pip install --upgrade samgeo

In [38]:
print("🎯 WORKING MODELS CONFIRMED:")
print(f"   ✅ YOLO: Available")
print(f"   ✅ SAM: Available")
print(f"   ❌ Transformers: Skipped (protobuf issue)")
print(f"   ❌ MediaPipe: Not available")

# Microtia guide parameters
BASE_THICKNESS = 2.0
HELIX_THICKNESS = 5.0
SKIN_TOLERANCE = 2.0
SUTURE_HOLE_SIZE = (1.2, 2.7)

print(f"\n📐 Surgical Guide Specifications:")
print(f"   Base thickness: {BASE_THICKNESS}mm")
print(f"   Helix thickness: {HELIX_THICKNESS}mm") 
print(f"   Skin tolerance: {SKIN_TOLERANCE}mm")
print(f"   Suture holes: {SUTURE_HOLE_SIZE[0]}x{SUTURE_HOLE_SIZE[1]}mm")

🎯 WORKING MODELS CONFIRMED:
   ✅ YOLO: Available
   ✅ SAM: Available
   ❌ Transformers: Skipped (protobuf issue)
   ❌ MediaPipe: Not available

📐 Surgical Guide Specifications:
   Base thickness: 2.0mm
   Helix thickness: 5.0mm
   Skin tolerance: 2.0mm
   Suture holes: 1.2x2.7mm


In [39]:
import cv2
import torch
import os
import glob
import numpy as np
from pathlib import Path
import matplotlib.pyplot as plt
from scipy import ndimage
import math
from ultralytics import YOLO
from segment_anything import sam_model_registry, SamPredictor

class MicrotiaSurgicalGuide:
    def __init__(self, yolo_model_path):
        self.yolo_model_path = yolo_model_path
        self.yolo_model = None
        self.sam_predictor = None
        
        self.load_models()
        
        # Surgical guide parameters
        self.base_thickness = 2.0
        self.helix_thickness = 5.0
        self.skin_tolerance = 2.0
        self.suture_hole_size = (1.2, 2.7)
    
    def download_sam_checkpoint(self):
        """Download SAM checkpoint if needed"""
        import urllib.request
        sam_checkpoints = {
            "vit_b": "https://dl.fbaipublicfiles.com/segment_anything/sam_vit_b_01ec64.pth",
            "vit_l": "https://dl.fbaipublicfiles.com/segment_anything/sam_vit_l_0b3195.pth"
        }
        
        checkpoint_dir = "sam_checkpoints"
        os.makedirs(checkpoint_dir, exist_ok=True)
        
        # Try to find existing checkpoint
        for model_type, url in sam_checkpoints.items():
            checkpoint_path = os.path.join(checkpoint_dir, f"sam_{model_type}.pth")
            if os.path.exists(checkpoint_path):
                print(f"📁 Found existing SAM checkpoint: {checkpoint_path}")
                return checkpoint_path, model_type
        
        # Download vit_b (smallest) if none found
        print("📥 Downloading SAM checkpoint (vit_b)...")
        model_type = "vit_b"
        url = sam_checkpoints[model_type]
        checkpoint_path = os.path.join(checkpoint_dir, f"sam_{model_type}.pth")
        
        try:
            urllib.request.urlretrieve(url, checkpoint_path)
            print(f"✅ Downloaded SAM checkpoint: {checkpoint_path}")
            return checkpoint_path, model_type
        except Exception as e:
            print(f"❌ Failed to download SAM checkpoint: {e}")
            return None, None
    
    def load_models(self):
        """Load YOLO and SAM models"""
        # Load YOLO
        try:
            self.yolo_model = YOLO(self.yolo_model_path)
            print(f"✅ YOLO model loaded: {self.yolo_model_path}")
        except Exception as e:
            print(f"❌ Error loading YOLO model: {e}")
            return
        
        # Load SAM
        try:
            checkpoint_path, model_type = self.download_sam_checkpoint()
            if checkpoint_path and model_type:
                device = "cuda" if torch.cuda.is_available() else "cpu"
                sam = sam_model_registry[model_type](checkpoint=checkpoint_path)
                sam.to(device=device)
                self.sam_predictor = SamPredictor(sam)
                print(f"✅ SAM model loaded ({model_type}) on {device}")
            else:
                print("❌ Could not load SAM model")
        except Exception as e:
            print(f"❌ Error loading SAM model: {e}")
    
    def yolo_sam_hybrid_segment(self, image_path, conf=0.3):
        """
        Hybrid segmentation: YOLO for detection + SAM for precise segmentation
        """
        print(f"   🔍 Running YOLO+SAM hybrid segmentation...")
        
        # Step 1: YOLO detection
        yolo_results = self.yolo_model.predict(image_path, imgsz=640, conf=conf, verbose=False)
        
        if len(yolo_results) == 0 or yolo_results[0].boxes is None:
            print("   ❌ YOLO: No ears detected")
            return None
        
        # Get bounding box from YOLO
        bbox = yolo_results[0].boxes.xyxy[0].cpu().numpy()
        print(f"   📦 YOLO detected ear with bbox: {bbox}")
        
        # Step 2: SAM segmentation with YOLO bbox as prompt
        image = cv2.imread(image_path)
        image_rgb = cv2.cvtColor(image, cv2.COLOR_BGR2RGB)
        
        try:
            self.sam_predictor.set_image(image_rgb)
            
            # Use YOLO bbox as prompt for SAM
            input_box = np.array(bbox)
            masks, scores, logits = self.sam_predictor.predict(
                point_coords=None,
                point_labels=None,
                box=input_box[None, :],
                multimask_output=False,  # Get single best mask
            )
            
            best_mask = masks[0]
            sam_mask = (best_mask * 255).astype(np.uint8)
            
            print(f"   ✅ SAM: Generated high-quality mask (score: {scores[0]:.3f})")
            return sam_mask
            
        except Exception as e:
            print(f"   ❌ SAM segmentation failed: {e}")
            # Fallback to YOLO mask
            if yolo_results[0].masks is not None:
                yolo_mask = yolo_results[0].masks[0].data[0].cpu().numpy()
                yolo_mask = (yolo_mask * 255).astype(np.uint8)
                print("   🔄 Using YOLO mask as fallback")
                return yolo_mask
            return None
    
    def preprocess_mask(self, mask):
        """Clean and SMOOTH mask for 3D modeling (FR-05)"""
        if mask.dtype != np.uint8:
            mask = (mask * 255).astype(np.uint8)
        
        # 1. Upscale for better resolution before smoothing (2x)
        mask = cv2.resize(mask, None, fx=2, fy=2, interpolation=cv2.INTER_CUBIC)
        
        # 2. Threshold
        _, binary_mask = cv2.threshold(mask, 127, 255, cv2.THRESH_BINARY)
        
        # 3. Clean noise (Morphological Open/Close)
        kernel = np.ones((5,5), np.uint8) # Increased kernel size
        cleaned_mask = cv2.morphologyEx(binary_mask, cv2.MORPH_CLOSE, kernel)
        cleaned_mask = cv2.morphologyEx(cleaned_mask, cv2.MORPH_OPEN, kernel)
        
        # 4. GAUSSIAN BLUR (The key to removing pixels)
        # Blur the edges, then threshold back to crisp lines
        blurred = cv2.GaussianBlur(cleaned_mask, (15, 15), 0)
        _, smooth_mask = cv2.threshold(blurred, 127, 255, cv2.THRESH_BINARY)
        
        # 5. Keep only largest contour
        contours, _ = cv2.findContours(smooth_mask, cv2.RETR_EXTERNAL, cv2.CHAIN_APPROX_SIMPLE)
        final_mask = np.zeros_like(smooth_mask)
        if contours:
            largest_contour = max(contours, key=cv2.contourArea)
            cv2.fillPoly(final_mask, [largest_contour], 255)
            
        # 6. Downscale back to original (optional, or keep high res for better 3D)
        # We will keep it 2x because it makes the 3D model smoother
        
        return final_mask
    
    def create_contours_with_tolerance(self, mask, tolerance_mm, pixels_per_mm=10):
        """Create outer and inner contours with skin tolerance"""
        contours, _ = cv2.findContours(mask, cv2.RETR_EXTERNAL, cv2.CHAIN_APPROX_SIMPLE)
        if not contours:
            return None, None
            
        main_contour = max(contours, key=cv2.contourArea)
        tolerance_pixels = int(tolerance_mm * pixels_per_mm)
        
        # Create outer contour (expanded for skin tolerance)
        outer_contour = self.offset_contour(main_contour, tolerance_pixels)
        
        return outer_contour, main_contour  # outer=base, inner=helix
    
    def offset_contour(self, contour, offset_pixels):
        """Offset contour by specified distance"""
        if offset_pixels == 0:
            return contour
            
        x, y, w, h = cv2.boundingRect(contour)
        padding = abs(offset_pixels) + 10
        
        mask = np.zeros((h + 2 * padding, w + 2 * padding), dtype=np.uint8)
        contour_shifted = contour - [x, y] + [padding, padding]
        cv2.fillPoly(mask, [contour_shifted], 255)
        
        kernel_size = abs(offset_pixels) * 2 + 1
        kernel = np.ones((kernel_size, kernel_size), np.uint8)
        
        if offset_pixels > 0:
            processed_mask = cv2.dilate(mask, kernel, iterations=1)
        else:
            processed_mask = cv2.erate(mask, kernel, iterations=1)
            
        new_contours, _ = cv2.findContours(processed_mask, cv2.RETR_EXTERNAL, cv2.CHAIN_APPROX_SIMPLE)
        if new_contours:
            new_contour = max(new_contours, key=cv2.contourArea)
            return new_contour + [x - padding, y - padding]
        return contour
    
    def find_curvature_points(self, contour, num_points=2):
        """Find high curvature points for suture holes"""
        if contour is None or len(contour) < 3:
            return []
            
        epsilon = 0.02 * cv2.arcLength(contour, True)
        approx_contour = cv2.approxPolyDP(contour, epsilon, True)
        points = approx_contour.reshape(-1, 2)
        
        if len(points) < 3:
            return []
        
        curvatures = []
        for i in range(len(points)):
            p1 = points[i-1]
            p2 = points[i]
            p3 = points[(i+1) % len(points)]
            
            v1 = p1 - p2
            v2 = p3 - p2
            
            dot_product = np.dot(v1, v2)
            norm_v1 = np.linalg.norm(v1)
            norm_v2 = np.linalg.norm(v2)
            
            if norm_v1 == 0 or norm_v2 == 0:
                continue
                
            cosine = dot_product / (norm_v1 * norm_v2)
            cosine = np.clip(cosine, -1.0, 1.0)
            angle = np.arccos(cosine)
            curvature = np.pi - angle  # Higher for sharper angles
            
            curvatures.append((p2[0], p2[1], curvature))
        
        if not curvatures:
            return []
            
        # Sort by curvature and take top points
        curvatures.sort(key=lambda x: x[2], reverse=True)
        suture_points = [(x, y) for x, y, curvature in curvatures[:num_points]]
        
        return suture_points

print("✅ MicrotiaSurgicalGuide class defined!")

# Fix for STL_AVAILABLE
try:
    from stl import mesh
    STL_AVAILABLE = True
    print("✅ numpy-stl imported successfully. 3D generation is ENABLED.")
except ImportError:
    STL_AVAILABLE = False
    print("❌ numpy-stl not found. 3D generation will be SKIPPED.")
    print("   Run: !pip install numpy-stl")

print(f"✅ Torch imported: {torch.__version__}")
print(f"✅ Device ready: {'cuda' if torch.cuda.is_available() else 'cpu'}")

✅ MicrotiaSurgicalGuide class defined!
✅ numpy-stl imported successfully. 3D generation is ENABLED.
✅ Torch imported: 2.0.1+cu117
✅ Device ready: cuda


In [40]:
import numpy as np
from stl import mesh
import cv2
import math
from scipy.interpolate import splprep, splev

# --- 1. GEOMETRY HELPERS (SMOOTHING & FACES) ---

def resample_contour(contour, n_points=500, smooth_factor=0.01):
    """
    Uses B-Spline interpolation to turn a jagged contour into a smooth, organic curve.
    Matches FR-05 (Contour Smoothing) requirements.
    """
    if len(contour.shape) > 2:
        contour = contour.squeeze()
        
    # Close the contour loop
    if not np.array_equal(contour[0], contour[-1]):
        contour = np.vstack((contour, contour[0]))
    
    # Need at least 4 points for cubic spline
    if len(contour) < 4:
        return contour
        
    x, y = contour.T
    try:
        # Create Spline (k=3 cubic, s=smoothness)
        tck, u = splprep([x, y], u=None, s=smooth_factor * len(contour), per=1, k=3)
        # Generate new even points
        u_new = np.linspace(u.min(), u.max(), n_points)
        x_new, y_new = splev(u_new, tck, der=0)
        return np.column_stack((x_new, y_new)).astype(np.float32)
    except Exception:
        return contour

def create_strip_faces(n_points, start_idx_A, start_idx_B):
    """Connect two rings of points with triangles"""
    faces = []
    for i in range(n_points - 1):
        faces.append([start_idx_A + i, start_idx_B + i, start_idx_A + i + 1])
        faces.append([start_idx_B + i, start_idx_B + i + 1, start_idx_A + i + 1])
    # Close loop
    faces.append([start_idx_A + n_points - 1, start_idx_B + n_points - 1, start_idx_A])
    faces.append([start_idx_B + n_points - 1, start_idx_B, start_idx_A])
    return faces

# --- 2. SUTURE HOLE GENERATOR (NEW) ---

def create_suture_holes_stl(suture_points, output_path, thickness=10.0, hole_radius=0.8):
    """
    Generates a separate STL of 'Drill Bits' to subtract from the main guide.
    """
    if not suture_points: return False
    print(f"   🕳️ Generating Suture Hole Cutters ({len(suture_points)} holes)...")
    
    all_faces = []
    all_vertices = []
    segments = 12 # Cylinder smoothness
    
    for cx, cy in suture_points:
        start_idx = len(all_vertices)
        # Bottom circle
        for i in range(segments):
            angle = 2 * math.pi * i / segments
            all_vertices.append([cx + hole_radius*math.cos(angle), cy + hole_radius*math.sin(angle), -5])
        # Top circle
        for i in range(segments):
            angle = 2 * math.pi * i / segments
            all_vertices.append([cx + hole_radius*math.cos(angle), cy + hole_radius*math.sin(angle), thickness+5])
            
        # Faces
        for i in range(segments):
            next_i = (i + 1) % segments
            all_faces.append([start_idx+i, start_idx+next_i, start_idx+i+segments])
            all_faces.append([start_idx+next_i, start_idx+next_i+segments, start_idx+i+segments])

    # Save STL
    hole_mesh = mesh.Mesh(np.zeros(len(all_faces), dtype=mesh.Mesh.dtype))
    for i, face in enumerate(all_faces):
        for j in range(3):
            hole_mesh.vectors[i][j] = all_vertices[face[j]]
    hole_mesh.save(output_path)
    return True

# --- 3. MAIN HOLLOW GUIDE GENERATOR ---

def create_hollow_guide_model(outer_contour, main_contour, output_path, 
                              base_thickness=2.0, helix_thickness=5.0, 
                              wall_width_mm=3.0, scale_factor=0.1):
    """Generates the Smooth Hollow Surgical Guide"""
    if 'STL_AVAILABLE' in globals() and not STL_AVAILABLE: return False
    print(f"   📐 Generating SMOOTH HOLLOW guide...")
    
    try:
        # A. Create Inner Void Contour
        pixels_per_mm = 10 
        wall_offset_pixels = int(wall_width_mm * pixels_per_mm)
        
        # Calculate Offset
        M = cv2.moments(main_contour)
        cX, cY = (int(M["m10"]/M["m00"]), int(M["m01"]/M["m00"])) if M["m00"] != 0 else (1000,1000)
        
        mask_h, mask_w = 2000, 2000
        temp_mask = np.zeros((mask_h, mask_w), dtype=np.uint8)
        shift_x, shift_y = 1000-cX, 1000-cY
        cv2.fillPoly(temp_mask, [main_contour + [shift_x, shift_y]], 255)
        
        kernel = np.ones((wall_offset_pixels*2+1, wall_offset_pixels*2+1), np.uint8)
        eroded_mask = cv2.erode(temp_mask, kernel, iterations=1)
        
        inner_contours, _ = cv2.findContours(eroded_mask, cv2.RETR_EXTERNAL, cv2.CHAIN_APPROX_SIMPLE)
        inner_void_contour = (max(inner_contours, key=cv2.contourArea) - [shift_x, shift_y]) if inner_contours else main_contour

        # B. Resample & Smooth All Contours
        N_POINTS = 500
        c_base  = resample_contour(outer_contour, N_POINTS, smooth_factor=2.0) * scale_factor
        c_helix = resample_contour(main_contour, N_POINTS, smooth_factor=0.5) * scale_factor
        c_void  = resample_contour(inner_void_contour, N_POINTS, smooth_factor=0.5) * scale_factor
        
        # C. Build Mesh Layers
        all_vertices, all_faces = [], []
        
        # Helper to add ring
        def add_ring(contour, z_height):
            start = len(all_vertices)
            for p in contour: all_vertices.append([p[0], p[1], z_height])
            return start

        # Layer 1: Base Flange
        idx_base_bot = add_ring(c_base, 0)
        idx_base_top = add_ring(c_base, base_thickness)
        idx_helix_bot = add_ring(c_helix, 0)
        idx_helix_top = add_ring(c_helix, base_thickness)
        
        all_faces.extend(create_strip_faces(N_POINTS, idx_base_bot, idx_helix_bot)) # Floor
        all_faces.extend(create_strip_faces(N_POINTS, idx_base_top, idx_helix_top)) # Ceiling
        all_faces.extend(create_strip_faces(N_POINTS, idx_base_bot, idx_base_top))  # Outer Wall
        
        # Layer 2: Helix Wall
        total_h = base_thickness + helix_thickness
        idx_helix_high = add_ring(c_helix, total_h)
        idx_void_bot   = add_ring(c_void, 0)
        idx_void_high  = add_ring(c_void, total_h)
        
        all_faces.extend(create_strip_faces(N_POINTS, idx_helix_top, idx_helix_high)) # Outer Vert
        all_faces.extend(create_strip_faces(N_POINTS, idx_helix_high, idx_void_high)) # Top Rim
        all_faces.extend(create_strip_faces(N_POINTS, idx_void_high, idx_void_bot))   # Inner Vert
        all_faces.extend(create_strip_faces(N_POINTS, idx_helix_bot, idx_void_bot))   # Bottom Seal

        # Save
        guide_mesh = mesh.Mesh(np.zeros(len(all_faces), dtype=mesh.Mesh.dtype))
        for i, face in enumerate(all_faces):
            for j in range(3): guide_mesh.vectors[i][j] = all_vertices[face[j]]
        guide_mesh.save(output_path)
        print(f"   ✅ Smooth Hollow Guide saved: {output_path}")
        return True
    except Exception as e:
        print(f"   ❌ Error: {e}")
        return False

def create_guide_visualization(original_mask, outer_contour, inner_contour, suture_points, output_path):
    try:
        vis = cv2.cvtColor(original_mask, cv2.COLOR_GRAY2BGR)
        cv2.drawContours(vis, [outer_contour], -1, (128,0,128), 3)
        cv2.drawContours(vis, [inner_contour], -1, (0,255,255), 3)
        for x,y in suture_points: cv2.rectangle(vis, (int(x)-8, int(y)-4), (int(x)+8, int(y)+4), (0,0,255), -1)
        cv2.imwrite(output_path.replace('.stl', '_visualization.jpg'), vis)
        return True
    except: return False

In [41]:
import cv2
import numpy as np

def extract_cartilage_skeleton(image_path, binary_mask_path):
    """
    Implements a 'Canny Edge' approach (common in literature) to find ear contours.
    1. Uses the AI mask to isolate the ear.
    2. Applies Canny Edge Detection to find the steep slopes of the cartilage.
    3. Filters for the longest continuous curves (Helix and Anti-Helix).
    """
    # 1. Load Original Image & Mask
    img = cv2.imread(image_path)
    mask = cv2.imread(binary_mask_path, cv2.IMREAD_GRAYSCALE)
    
    if img is None or mask is None: return None
    
    # 2. Preprocessing (Standard in literature)
    # Mask the image to remove hair/background noise
    ear_zone = cv2.bitwise_and(img, img, mask=mask)
    
    # Convert to Grayscale
    gray = cv2.cvtColor(ear_zone, cv2.COLOR_BGR2GRAY)
    
    # Histogram Equalization (CLAHE) to fix lighting differences
    clahe = cv2.createCLAHE(clipLimit=2.0, tileGridSize=(8,8))
    enhanced = clahe.apply(gray)
    
    # Strong Gaussian Blur to ignore pores/skin texture
    # We only want the major ridges
    blurred = cv2.GaussianBlur(enhanced, (9, 9), 0)
    
    # 3. Canny Edge Detection (The Paper's Method)
    # Finds the "edges" where the cartilage rises from the skin
    # Thresholds: 30 (low) and 100 (high) are standard starting points for ears
    edges = cv2.Canny(blurred, 30, 100)
    
    # 4. Post-Processing to Connect Gaps (Morphology)
    # Dilate edges slightly to connect broken lines
    kernel = np.ones((3,3), np.uint8)
    dilated_edges = cv2.dilate(edges, kernel, iterations=1)
    
    # 5. Contour Filtering (The "Complex" Extraction)
    # We only want the BIG curves (Helix + Anti-Helix), not small noise
    contours, _ = cv2.findContours(dilated_edges, cv2.RETR_EXTERNAL, cv2.CHAIN_APPROX_SIMPLE)
    
    clean_skeleton = np.zeros_like(mask)
    
    if contours:
        # Sort contours by length (arcLength), longest first
        sorted_cnts = sorted(contours, key=lambda x: cv2.arcLength(x, False), reverse=True)
        
        # Keep the Top 3 longest curves
        # Usually: 1. Outer Helix, 2. Anti-Helix (Y-shape), 3. Tragus/Lobe
        for i in range(min(5, len(sorted_cnts))):
            cnt = sorted_cnts[i]
            if cv2.contourArea(cnt) > 50 or cv2.arcLength(cnt, False) > 100:
                # Draw thick lines to ensure 3D printer wall thickness
                cv2.drawContours(clean_skeleton, [cnt], -1, 255, thickness=15) 
                
    # 6. Merge with Outer Frame
    # We always need the outer solid shape (Base Plate) + The new Inner Skeleton
    # Convert original mask to a simple outline
    outer_contours, _ = cv2.findContours(mask, cv2.RETR_EXTERNAL, cv2.CHAIN_APPROX_SIMPLE)
    if outer_contours:
        main_blob = max(outer_contours, key=cv2.contourArea)
        # Draw the base plate outline
        cv2.drawContours(clean_skeleton, [main_blob], -1, 255, thickness=10)
        
    # Final close to ensure watertightness
    clean_skeleton = cv2.morphologyEx(clean_skeleton, cv2.MORPH_CLOSE, kernel, iterations=2)
    
    return clean_skeleton

print("✅ Canny Edge Logic (Literature Based) Defined")

✅ Canny Edge Logic (Literature Based) Defined


In [42]:
def create_microtia_guide(image_path, output_dir="surgical_guides", conf=0.3):
    """Pipeline: Image -> Ridge Skeleton -> Hollow Guide + Suture Holes"""
    
    # Init Model
    model_path = r"A:\PROJECT\AUTO MICROTIA\Ear-segmentation-ai\ear_segmentation\models\best_v4.pt"
    guide_generator = MicrotiaSurgicalGuide(model_path)
    if guide_generator.yolo_model is None: return None
    
    os.makedirs(output_dir, exist_ok=True)
    base_name = Path(image_path).stem
    print(f"\n🎯 PROCESSING: {base_name}")
    print("="*50)
    
    # 1. Segmentation (Get the Blob)
    hybrid_mask = guide_generator.yolo_sam_hybrid_segment(image_path, conf=conf)
    if hybrid_mask is None: return None
    
    blob_mask_path = os.path.join(output_dir, f"{base_name}_blob_mask.png")
    cv2.imwrite(blob_mask_path, hybrid_mask)
    
    # 2. RIDGE DETECTION (The Fix for 'Completely Different')
    # Use the blob mask + original image to find the skeleton
    skeleton_mask = extract_cartilage_skeleton(image_path, blob_mask_path)
    
    if skeleton_mask is None:
        print("⚠️ Ridge detection failed, falling back to blob.")
        skeleton_mask = guide_generator.preprocess_mask(hybrid_mask)
    
    # Save the skeleton mask so you can verify it matches the reference photo
    skel_path = os.path.join(output_dir, f"{base_name}_skeleton_mask.png")
    cv2.imwrite(skel_path, skeleton_mask)
    print(f"   💀 Skeleton Mask generated (Check {skel_path})")

    # 3. Contours (Using the new Skeleton Mask)
    # We process the skeleton mask to smooth it out
    processed_mask = guide_generator.preprocess_mask(skeleton_mask)
    
    outer_contour, inner_contour = guide_generator.create_contours_with_tolerance(
        processed_mask, guide_generator.skin_tolerance, pixels_per_mm=10
    )
    if outer_contour is None: return None
    
    # 4. Centerline & Suture Points
    pixels_per_mm = 10
    half_wall_pixels = int(1.5 * pixels_per_mm)
    
    temp_mask = np.zeros_like(processed_mask)
    M = cv2.moments(inner_contour)
    if M["m00"] != 0:
        cX, cY = int(M["m10"]/M["m00"]), int(M["m01"]/M["m00"])
        shift_x, shift_y = temp_mask.shape[1]//2 - cX, temp_mask.shape[0]//2 - cY
        shifted_contour = inner_contour + [shift_x, shift_y]
        
        temp_canvas = np.zeros_like(processed_mask)
        cv2.fillPoly(temp_canvas, [shifted_contour], 255)
        kernel = np.ones((half_wall_pixels*2+1, half_wall_pixels*2+1), np.uint8)
        eroded_canvas = cv2.erode(temp_canvas, kernel, iterations=1)
        mid_contours, _ = cv2.findContours(eroded_canvas, cv2.RETR_EXTERNAL, cv2.CHAIN_APPROX_SIMPLE)
        
        centerline_contour = (max(mid_contours, key=cv2.contourArea) - [shift_x, shift_y]) if mid_contours else inner_contour
    else:
        centerline_contour = inner_contour

    suture_points = guide_generator.find_curvature_points(centerline_contour)
    
    # 5. Generate Outputs
    guide_path = os.path.join(output_dir, f"{base_name}_microtia_guide.stl")
    holes_path = os.path.join(output_dir, f"{base_name}_suture_holes.stl")
    
    # Visualization
    try:
        vis_img = cv2.cvtColor(processed_mask, cv2.COLOR_GRAY2BGR)
        cv2.drawContours(vis_img, [outer_contour], -1, (128,0,128), 2)
        cv2.drawContours(vis_img, [inner_contour], -1, (0,255,255), 2)
        cv2.drawContours(vis_img, [centerline_contour], -1, (255,255,0), 1)
        for x,y in suture_points: cv2.rectangle(vis_img, (int(x)-6, int(y)-3), (int(x)+6, int(y)+3), (0,0,255), -1)
        cv2.imwrite(guide_path.replace('.stl', '_visualization.jpg'), vis_img)
        vis_success = True
    except: vis_success = False
    
    # 3D Models
    model_success = False
    if 'STL_AVAILABLE' in globals() and STL_AVAILABLE:
        model_success = create_hollow_guide_model(
            outer_contour,      
            inner_contour,      
            guide_path,
            base_thickness=2.0,
            helix_thickness=5.0,
            wall_width_mm=3.0
        )
        if model_success:
            create_suture_holes_stl(suture_points, holes_path)

    result = {
        'success': vis_success or model_success,
        'image_name': base_name,
        'mask_path': skel_path,
        'guide_path': guide_path,
        'holes_path': holes_path,
        'suture_points': len(suture_points),
        'segmentation_method': 'YOLO+SAM Hybrid (Skeleton Mode)'
    }
    
    if model_success:
        print(f"   ✨ Guide: {guide_path}")
        print(f"   ✨ Holes: {holes_path}")
        
    return result

In [43]:
def batch_process_with_comparison(input_dir="input", output_dir="surgical_guides", max_images=None):
    """Process all images and compare YOLO vs YOLO+SAM results"""
    if not os.path.exists(input_dir):
        print(f"❌ Input directory not found: {input_dir}")
        return
    
    image_files = glob.glob(os.path.join(input_dir, "*.jpg")) + \
                  glob.glob(os.path.join(input_dir, "*.png")) + \
                  glob.glob(os.path.join(input_dir, "*.jpeg"))
    
    if not image_files:
        print(f"❌ No images found in {input_dir}")
        return
    
    if max_images:
        image_files = image_files[:max_images]
    
    print(f"🔍 Found {len(image_files)} images")
    print("🚀 Starting batch processing with YOLO+SAM hybrid...")
    
    results = []
    successful_guides = 0
    
    for i, img_path in enumerate(image_files):
        print(f"\n" + "="*60)
        print(f"📸 Processing {i+1}/{len(image_files)}: {Path(img_path).name}")
        print("="*60)
        
        result = create_microtia_guide(img_path, output_dir)
        
        if result and result['success']:
            successful_guides += 1
            status = "✅ SUCCESS"
        else:
            status = "❌ FAILED"
        
        if result:
            print(f"   {status}")
            print(f"   Method: {result['segmentation_method']}")
            print(f"   Suture points: {result['suture_points']}")
            if result['guide_path']:
                print(f"   3D Guide: ✅ Generated")
            else:
                print(f"   3D Guide: ❌ Not generated")
        
        results.append(result)
    
    # Summary
    print(f"\n🎯 BATCH PROCESSING COMPLETE!")
    print(f"📊 Results: {successful_guides}/{len(image_files)} successful guides")
    print(f"📁 Output folder: {output_dir}")
    
    return results

# Run the batch processing
results = batch_process_with_comparison(
    input_dir="input",
    output_dir="surgical_guides",
    max_images=None  # Process all images
)

🔍 Found 5 images
🚀 Starting batch processing with YOLO+SAM hybrid...

📸 Processing 1/5: 2D EAR 1.jpg
✅ YOLO model loaded: A:\PROJECT\AUTO MICROTIA\Ear-segmentation-ai\ear_segmentation\models\best_v4.pt
📁 Found existing SAM checkpoint: sam_checkpoints\sam_vit_b.pth
✅ SAM model loaded (vit_b) on cuda

🎯 PROCESSING: 2D EAR 1
   🔍 Running YOLO+SAM hybrid segmentation...
   📦 YOLO detected ear with bbox: [     496.27      291.01      1226.2      1141.4]
   ✅ SAM: Generated high-quality mask (score: 0.838)
   💀 Skeleton Mask generated (Check surgical_guides\2D EAR 1_skeleton_mask.png)
   📐 Generating SMOOTH HOLLOW guide...
   ✅ Smooth Hollow Guide saved: surgical_guides\2D EAR 1_microtia_guide.stl
   🕳️ Generating Suture Hole Cutters (2 holes)...
   ✨ Guide: surgical_guides\2D EAR 1_microtia_guide.stl
   ✨ Holes: surgical_guides\2D EAR 1_suture_holes.stl
   ✅ SUCCESS
   Method: YOLO+SAM Hybrid (Skeleton Mode)
   Suture points: 2
   3D Guide: ✅ Generated

📸 Processing 2/5: 2D EAR 2.jpg
✅ YOLO

In [44]:
import cv2
import numpy as np
import matplotlib.pyplot as plt
from pathlib import Path
import glob
import os

def generate_surgical_reference_card(image_path, mask_path, output_path):
    """
    Generates a 'Djoko Kuswanto Style' Surgical Planning Card.
    Robustly handles mask loading to prevent shape errors.
    """
    # 1. Load Image and Mask
    img = cv2.imread(image_path)
    # Load mask as-is first, then convert carefully
    mask = cv2.imread(mask_path)
    
    if img is None or mask is None:
        print(f"❌ Error loading files.\nImg: {image_path}\nMask: {mask_path}")
        return

    img = cv2.cvtColor(img, cv2.COLOR_BGR2RGB)
    
    # --- FIX FOR "TOO MANY VALUES TO UNPACK" ---
    # Ensure mask is strictly 1-channel Grayscale
    if len(mask.shape) == 3:
        mask = cv2.cvtColor(mask, cv2.COLOR_BGR2GRAY)
    
    # Now this will work safely because shape is guaranteed to be (h, w)
    h, w = mask.shape
    # -------------------------------------------

    # 2. Extract Edges (The Rib Framework Outline)
    contours, _ = cv2.findContours(mask, cv2.RETR_EXTERNAL, cv2.CHAIN_APPROX_SIMPLE)
    if not contours: 
        print("❌ No contours found in mask.")
        return
    main_contour = max(contours, key=cv2.contourArea)
    
    # Create a blank canvas for the template
    template = np.ones((h, w, 3), dtype=np.uint8) * 255 # White background
    
    # Draw the Base Plate (Gray) - Represents the 6th/7th Rib Base
    cv2.drawContours(template, [main_contour], -1, (220, 220, 220), -1)
    cv2.drawContours(template, [main_contour], -1, (50, 50, 50), 2)
    
    # 3. IDENTIFY ANATOMICAL ZONES (The "Y" Shape)
    # We erode the mask deeply to find the "Core" (Anti-helix area)
    kernel = np.ones((15, 15), np.uint8)
    eroded = cv2.erode(mask, kernel, iterations=4)
    core_contours, _ = cv2.findContours(eroded, cv2.RETR_EXTERNAL, cv2.CHAIN_APPROX_SIMPLE)
    
    annotated_img = template.copy()
    
    suture_targets = []
    
    # Draw The "Relief" (8th Rib Piece) - The Anti-Helix
    if core_contours:
        core_cnt = max(core_contours, key=cv2.contourArea)
        cv2.drawContours(annotated_img, [core_cnt], -1, (255, 200, 200), -1) # Light Blue/Red fill
        cv2.drawContours(annotated_img, [core_cnt], -1, (255, 0, 0), 2)     # Blue Outline
        
        # Calculate centroids for hole placement
        # Top of the Y (Superior Crus)
        try:
            top_y_idx = core_cnt[:,:,1].argmin()
            top_y = tuple(core_cnt[top_y_idx][0])
            suture_targets.append((top_y, "Superior Crus\n(Anchor 1)"))
        except: pass
        
        # Middle of Y (Anti-Helix Body)
        M = cv2.moments(core_cnt)
        if M["m00"] != 0:
            cx, cy = int(M["m10"]/M["m00"]), int(M["m01"]/M["m00"])
            suture_targets.append(((cx, cy), "Anti-Helix Body\n(Anchor 2)"))
            
        # Bottom Tail
        try:
            bot_y_idx = core_cnt[:,:,1].argmax()
            bot_y = tuple(core_cnt[bot_y_idx][0])
            suture_targets.append((bot_y, "Cauda Helix\n(Anchor 3)"))
        except: pass

    # 4. PLOT THE REFERENCE CARD
    plt.figure(figsize=(12, 8))
    
    # Background Grid
    plt.imshow(annotated_img)
    plt.grid(True, which='both', linestyle='--', linewidth=0.5, alpha=0.3)
    
    # Plot Suture Targets
    for (x, y), label in suture_targets:
        plt.scatter(x, y, c='red', s=150, edgecolors='white', linewidth=2, zorder=5)
        plt.text(x + 20, y, label, color='darkred', fontsize=10, fontweight='bold',
                 bbox=dict(facecolor='white', alpha=0.8, edgecolor='none'))

    # Plot Outer Helix Anchors (Manually estimated from contour)
    try:
        outer_points = main_contour[:, 0, :]
        right_idx = outer_points[:, 0].argmax()
        rx, ry = outer_points[right_idx]
        
        plt.scatter(rx - 30, ry, c='green', s=150, edgecolors='white', linewidth=2, zorder=5)
        plt.text(rx, ry, "Helix Rim\n(Stabilizer)", color='darkgreen', fontsize=10, fontweight='bold',
                 bbox=dict(facecolor='white', alpha=0.8, edgecolor='none'))
    except: pass

    # Titles and Legends
    plt.title("SURGICAL PRE-OPERATIVE PLAN: SUTURE FIXATION MAP", fontsize=14, fontweight='bold', pad=20)
    plt.xlabel("Grid Scale: 10px (~1mm)", fontsize=8)
    
    textstr = '\n'.join((
        r'$\bf{Technique:}$ Nagata 2-Stage',
        r'$\bf{Red\ Zone:}$ 8th Rib (Anti-Helix)',
        r'$\bf{Gray\ Zone:}$ 6th/7th Rib (Base)',
        r'$\bf{Dots:}$ Recommended Wire Points'
    ))
    props = dict(boxstyle='round', facecolor='wheat', alpha=0.5)
    plt.text(0.05, 0.95, textstr, transform=plt.gca().transAxes, fontsize=10,
            verticalalignment='top', bbox=props)

    plt.axis('off')
    
    # Save
    print(f"   💾 Saving Surgical Reference Card: {output_path}")
    plt.savefig(output_path, dpi=150, bbox_inches='tight')
    plt.show()

# --- EXECUTE ---
processed_masks = glob.glob("surgical_guides/*_segmentation_mask.png")
original_images = glob.glob("input/*.jpg") + glob.glob("input/*.png")

if processed_masks:
    mask_file = processed_masks[0]
    # Simple matching strategy
    base_name = Path(mask_file).name.replace("_segmentation_mask.png", "")
    
    # Try to find matching image
    matching_inputs = [f for f in original_images if base_name in f]
    
    if matching_inputs:
        img_file = matching_inputs[0]
        output_png = os.path.join("surgical_guides", f"{base_name}_SURGICAL_PLAN.png")
        generate_surgical_reference_card(img_file, mask_file, output_png)
    else:
        # Fallback: Just use the first image available if name match fails
        if original_images:
            print(f"⚠️ Exact name match not found. Using {original_images[0]} as reference background.")
            output_png = os.path.join("surgical_guides", f"{base_name}_SURGICAL_PLAN.png")
            generate_surgical_reference_card(original_images[0], mask_file, output_png)
else:
    print("❌ No processed masks found. Please run the generation pipeline first.")

❌ No processed masks found. Please run the generation pipeline first.


In [45]:
def generate_processing_summary(results, output_dir="surgical_guides"):
    """Generate comprehensive processing summary"""
    import csv
    from datetime import datetime
    
    summary_path = os.path.join(output_dir, "processing_summary.csv")
    
    # Collect statistics
    successful_guides = sum(1 for r in results if r and r['success'])
    total_images = len(results)
    
    # Write CSV summary
    with open(summary_path, 'w', newline='', encoding='utf-8') as f:
        writer = csv.writer(f)
        writer.writerow(['Image', 'Status', 'Method', 'Suture_Points', '3D_Guide', 'Mask_Generated'])
        
        for result in results:
            if result:
                writer.writerow([
                    result['image_name'],
                    'SUCCESS' if result['success'] else 'FAILED',
                    result['segmentation_method'],
                    result['suture_points'],
                    'YES' if result['guide_path'] else 'NO',
                    'YES' if result['mask_path'] else 'NO'
                ])
            else:
                writer.writerow(['Unknown', 'FAILED', 'Unknown', 0, 'NO', 'NO'])
    
    # Print summary
    print(f"\n📊 COMPREHENSIVE SUMMARY")
    print("="*40)
    print(f"Total images processed: {total_images}")
    print(f"Successful guides: {successful_guides}")
    print(f"Success rate: {(successful_guides/total_images)*100:.1f}%" if total_images > 0 else "0%")
    print(f"Summary file: {summary_path}")
    
    # Check STL files
    stl_files = glob.glob(os.path.join(output_dir, "*.stl"))
    print(f"3D STL files generated: {len(stl_files)}")
    
    # Quality assessment
    if successful_guides > 0:
        print(f"\n✅ PROCESSING SUCCESSFUL!")
        print("💡 Check the 'surgical_guides' folder for:")
        print("   - Segmentation masks")
        print("   - 3D surgical guide STL files")
        print("   - Guide visualizations")
        print("   - Processing summary")
    else:
        print(f"\n❌ PROCESSING FAILED")
        print("💡 Check the error messages above for troubleshooting")
    
    return summary_path

# Generate summary
if results:
    summary_file = generate_processing_summary(results)
    print(f"\n🎉 MICROTIA SURGICAL GUIDE GENERATION COMPLETED!")
else:
    print("❌ No results to summarize")


📊 COMPREHENSIVE SUMMARY
Total images processed: 5
Successful guides: 5
Success rate: 100.0%
Summary file: surgical_guides\processing_summary.csv
3D STL files generated: 10

✅ PROCESSING SUCCESSFUL!
💡 Check the 'surgical_guides' folder for:
   - Segmentation masks
   - 3D surgical guide STL files
   - Guide visualizations
   - Processing summary

🎉 MICROTIA SURGICAL GUIDE GENERATION COMPLETED!
